In [1]:
pip install pulp

Note: you may need to restart the kernel to use updated packages.


This model was generated to do an initial test on initial players to choose from FPL historical data

This needs to be added to the main FPL note book ultimately so there is one source of information

In [2]:
import requests
import pulp as lp

# Define the FPL API URL for player data
fpl_api_url = "https://fantasy.premierleague.com/api/bootstrap-static/"

# Make a request to the FPL API to fetch player data
response = requests.get(fpl_api_url)
data = response.json()

# Extract player data from the API response
players_data = {}
teams_data = {}  # Store team names
for player in data["elements"]:
    player_name = f"{player['first_name']} {player['second_name']}"
    players_data[player_name] = {
        "position": player["element_type"],
        "points_total": player["total_points"],
        "cost": player["now_cost"] / 10,  # Convert cost to millions
        "team": player["team"],
    }
    team_id = player["team"]
    team_name = data["teams"][team_id - 1]["name"]  # Adjust team_id to index
    teams_data[team_id] = team_name

# Define your total budget in millions
total_budget = 100  # Updated maximum budget to £100 million

# Define the maximum and minimum number of players allowed for each position
max_players_per_position = {
    1: 2,  # Goalkeeper
    2: 5,  # Defender
    3: 5,  # Midfielder
    4: 3,  # Forward (Striker)
}

min_players_per_position = {
    1: 1,  # Minimum 1 Goalkeeper
    2: 3,  # Minimum 3 Defenders
    3: 3,  # Minimum 3 Midfielders
    4: 1,  # Minimum 1 Forward (Striker)
}

# Define the minimum total players required
min_total_players = 13  # Updated minimum team size to 13 players

# Create a PuLP linear programming problem
model = lp.LpProblem("FantasyFootballTeam", lp.LpMaximize)

# Create a binary variable for each player to represent if they are in the team
player_vars = {player: lp.LpVariable(player, cat=lp.LpBinary) for player in players_data.keys()}

# Objective function: maximize total points
model += lp.lpSum(players_data[player]["points_total"] * player_vars[player] for player in players_data.keys()), "TotalPoints"

# Constraints
# Budget constraint
model += lp.lpSum(players_data[player]["cost"] * player_vars[player] for player in players_data.keys()) <= total_budget, "Budget"

# Position constraints
for position, max_count in max_players_per_position.items():
    model += lp.lpSum(player_vars[player] for player in players_data.keys() if players_data[player]["position"] == position) <= max_count, f"MaxPlayersPosition{position}"

# Minimum players per position constraints
for position, min_count in min_players_per_position.items():
    model += lp.lpSum(player_vars[player] for player in players_data.keys() if players_data[player]["position"] == position) >= min_count, f"MinPlayersPosition{position}"

# Minimum total players constraint
model += lp.lpSum(player_vars[player] for player in players_data.keys()) >= min_total_players, "MinTotalPlayers"

# Solve the ILP problem
model.solve()

# Create a dictionary to store selected players by position
selected_players_by_position = {
    1: [],  # Goalkeeper
    2: [],  # Defender
    3: [],  # Midfielder
    4: [],  # Forward (Striker)
}

# Populate the dictionary with selected players
for player in players_data.keys():
    if lp.value(player_vars[player]) > 0:
        data = players_data[player]
        selected_players_by_position[data["position"]].append(player)

# Print the selected players and their data sorted by position
for position in selected_players_by_position.keys():
    position_name = {
        1: "Goalkeeper",
        2: "Defender",
        3: "Midfielder",
        4: "Forward (Striker)",
    }.get(position, "Unknown")
    print(f"Position: {position_name}")
    for player in selected_players_by_position[position]:
        data = players_data[player]
        team_name = teams_data[data["team"]]  # Get team name from teams_data
        print(f"Player: {player} - Points: {data['points_total']} - Cost: £{data['cost']} million - Team: {team_name}")

# Suggest the captain based on the player with the highest points
captain = None
max_points = 0
for player in players_data.keys():
    if lp.value(player_vars[player]) > 0:
        if players_data[player]["points_total"] > max_points:
            captain = player
            max_points = players_data[player]["points_total"]

if captain:
    print(f"\nSuggested Captain (Double Points): {captain}")

# You can customize the constraints and objective function as needed


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/codespace/.python/current/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/d9bf6648aab6451aa8787173383f2b13-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/d9bf6648aab6451aa8787173383f2b13-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 15 COLUMNS
At line 4779 RHS
At line 4790 BOUNDS
At line 5513 ENDATA
Problem MODEL has 10 rows, 722 columns and 2888 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 636.306 - 0.00 seconds
Cgl0003I 0 fixed, 3 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 6 rows, 407 columns (407 integer (323 of which binary)) and 1221 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 2 integers unsatisfied sum - 0.494118
Cbc0038I Pass   1: suminf.    0.14894 (1) 